<a href="https://colab.research.google.com/github/adelacvg/ttts/blob/v4/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !git clone -b v4 https://github.com/adelacvg/ttts.git

In [ ]:
!pip install -e .

In [ ]:
!git lfs install
!git clone https://huggingface.co/adelacvg/TTTS_v4

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install pypinyin einops omegaconf faiss-cpu==1.8.0 cutlet hangul_romanize unidic
!python -m unidic download

In [ ]:
from pypinyin import lazy_pinyin, Style
import torch
import numpy as np
from ttts.utils import vc_utils
import cutlet
from hangul_romanize import Transliter
from hangul_romanize.rule import academic
import torchaudio
from pypinyin import pinyin, lazy_pinyin, Style

device = 'cuda:0'
MODELS = {
    # "vqvae.pth": "model-520.pt",
    "vqvae.pth": "model-1060.pt"
}


speaker = 'unseen3'
cond_audio = "wenetdemo/Ref/" + speaker + "_spk.wav"
lang = 'ZH'
variable_texts = {
    "text1": "应该给千千万万还在路上的创业者致意。",
    "text2": "准确点说，小森林是一部美食类电影食物佳肴，贯穿了柿子的寒暑交替四十三餐。",
    "text3": "要大力开展全省网吧专项整治工作，加强网络文化内容的整治，深入开展低俗音像制品清查行动。",
    "text4": "因为这是我们法律存在的前提。",
    "text5": "书记旗县市长乡镇苏木长企事业负责人以及国营农牧场的老兵团们，像赶庙会似的你挤我扛，口里说着，手里记着。",
    "text6": "哎，不是有一句话这样说嘛，你永远不知道明天和意外哪个先来。",
    "text7": "湿热天最麻烦的还属家中的煤气，早上起来，连拌果酱的木铲也发霉了。",
    "text8": "这时，朱警官等人才发现小男孩腿脚也异常，根本走不了路。",
    "text9": "他一度梦想参军，但是因没身份而作罢，整日在村内瞎晃。",
    "text10": "当时有点感动还是什么？",
    "text11": "这些飞机仍然由位于华盛顿州埃弗里特的工厂生产。",
    "text12": "派大军前去争抢，赫赫查拉二话不说，喊出了巴顿的名字，结果手套开始传递给斯科特半路遭到灭霸拦截，真正的杀神从天而降。",
    "text13": "而中国却只是将其作为红旗反导系统下的一个补充而已。",
    "text14": "这一段戏同样也表达了亚瑟说的，我原本以为我的人生是一出悲剧，但其实它是一出戏剧。",
    "text15": "常常暴跳如雷，红着眼睛用力攥着洛洛的手，把洛洛的手攥得生疼也不撒手。洛洛吓坏了，她很害怕，那个平日里对她那么好的男友为什么会变得这么可怕？",
}
text_choose = "text1"
text = variable_texts.get(text_choose)


def multi_lang_convert(text, lang):
    if lang == "ZH":
        text = ' '.join(lazy_pinyin(text, style=Style.TONE3, neutral_tone_with_five=True))
        text = ' '+text+' '
    else:
        return None
    return text


from ttts.gpt.voice_tokenizer import VoiceBpeTokenizer
import torch.nn.functional as F

tok_zh = VoiceBpeTokenizer('tokenizers/zh_tokenizer.json')
text = multi_lang_convert(text, lang)
if lang == "ZH":
    text = tok_zh.encode(text.lower())
else:
    print('not supprt lang')
text = torch.LongTensor(text).unsqueeze(0)
if lang == "ZH":
    text = text + 256*0
    lang = 0
text_tokens = text.to(device)
text_length = torch.LongTensor([text_tokens.shape[1]]).to(device)


from ttts.utils.infer_utils import load_model
import torchaudio
import json
import torchaudio.functional as F
from ttts.utils.data_utils import spectrogram_torch,HParams
hps = HParams(**json.load(open('ttts/vqvae/config_v3.json')))
wav, sr = torchaudio.load(cond_audio)
if wav.shape[0] > 1:
    wav = wav[0].unsqueeze(0)
wav = wav.to(device)
wav32k = F.resample(wav, sr, 32000)
wav32k = wav32k[:,:int(hps.data.hop_length * (wav32k.shape[-1]//hps.data.hop_length))]
wav = torch.clamp(wav32k, min=-1.0, max=1.0)
wav_length = torch.LongTensor([wav.shape[1]])
spec = spectrogram_torch(wav, hps.data.filter_length,
                    hps.data.hop_length, hps.data.win_length, center=False).squeeze(0)
spec_length = torch.LongTensor([
    x//hps.data.hop_length for x in wav_length]).to(device)
vqvae = load_model('vqvae', MODELS['vqvae.pth'], 'ttts/vqvae/config_v3.json', device)
with torch.no_grad():
    wav_out = vqvae.infer(text_tokens, text_length, spec.unsqueeze(0), spec_length)

save_name = "wenetdemo/" + speaker + "_" + text_choose + ".wav"
torchaudio.save(save_name, wav_out.squeeze(0).cpu(), 32000)

In [ ]:
from IPython.display import Audio
wav = wav_out.detach().cpu()
Audio(wav[0],rate=32000)